**Packages Installation**

In [2]:
# %cd /home/2023-1_DL_TeamProject_t5
# %pip install -r requirements.txt

/home/2023-1_DL_TeamProject_t5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 34.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 46.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 53.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.3/293.3 kB 42.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 45.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 54.0 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.0/719.0 kB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 50.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**Import**

In [1]:
import random
import pandas as pd
import numpy as np
import math
import os
from PIL import Image
from pathlib import Path
import re
# from nltk import edit_distance

from torch.nn.utils.rnn import pad_sequence
from torch.optim.lr_scheduler import LambdaLR

import pytorch_lightning as pl
from pytorch_lightning.utilities import rank_zero_only

import torch
from torch.utils.data import Dataset, DataLoader

from transformers import (
    DonutProcessor,
    VisionEncoderDecoderConfig,
    VisionEncoderDecoderModel,
    get_scheduler
)

import wandb

from tqdm.auto import tqdm

import warnings
warnings.filterwarnings(action='ignore') 

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device.type

'cuda'

**Hyperparameter Settings**

In [3]:
CFG = {
    'WORKING_DIR': "/home/2023-1_DL_TeamProject_t5",
    'SEED':42,
    'NUM_WORKERS':4,
    'IMG_HEIGHT':4032,
    'IMG_WIDTH':3024,
    'MAX_LEN':1024,
    'BATCH_SIZE':2
}

**Set Working Direcotry**

In [4]:
os.chdir(CFG['WORKING_DIR'])
print(os.getcwd())

/home/2023-1_DL_TeamProject_t5


**Fix Seeds**

In [5]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED'])

**Dataset Building & Data Preprocessing**

In [6]:
type_dict = {0:"uni", 1:"nm", 2:"ing", 3:"exp", 4:"how", 5:"des", 9:"etc"}

class DonutDataset(Dataset):

    def __init__(
        self,
        dataframe: pd.DataFrame,
        max_length: int,
        processor: DonutProcessor,
        split: str = "train",
        ignore_id: int = -100,
    ):
        super().__init__()

        self.max_length = max_length
        self.split = split
        self.ignore_id = ignore_id
        self.dataframe = dataframe
        self.dataframe_length = len(self.dataframe)
        self.processor = processor
        self.gt_container = []
        
        for idx, sample in self.dataframe.iterrows():
            ground_truth = self.get_gt_strings(eval(sample['texts']))
            self.gt_container.append(ground_truth)

    def get_gt_strings(self, ct):
        
        gt_string = ""
        flag = 1
        tp = -1
        for i, item in enumerate(ct):
            if flag:
                gt_string = gt_string + f'<{type_dict[item[0]]}>'
                tp = item[0]
                flag = 0
                gt_string = gt_string + f'{item[1]}'
            
            elif not flag:
                gt_string = gt_string + f' {item[1]}'
            
            if i == len(ct)-1 or ct[i+1][0] != tp:
                gt_string = gt_string + f'</{type_dict[item[0]]}>'
                flag = 1
        
        return gt_string
    
    def __len__(self):
        
        return self.dataframe_length

    def __getitem__(self, idx: int):

        sample = self.dataframe.loc[idx]
        image = Image.open(sample['image_path'])
       
        pixel_values = self.processor(image, random_padding=self.split == "train", return_tensors="pt").pixel_values.squeeze()

        target_sequence = self.gt_container[idx] 
        input_ids = self.processor.tokenizer(
            target_sequence,
            add_special_tokens=False,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )["input_ids"].squeeze(0)

        labels = input_ids.clone()
        labels[labels == self.processor.tokenizer.pad_token_id] = self.ignore_id  

        return pixel_values, labels, target_sequence

In [7]:
processor = DonutProcessor.from_pretrained("naver-clova-ix/donut-base")
processor.image_processor.size = {"height": CFG['IMG_HEIGHT'],"width": CFG['IMG_WIDTH']}

config = VisionEncoderDecoderConfig.from_pretrained("naver-clova-ix/donut-base")
config.encoder.image_size = [CFG['IMG_HEIGHT'], CFG['IMG_WIDTH']]
config.decoder.max_length = CFG['MAX_LEN']

model = VisionEncoderDecoderModel.from_pretrained("naver-clova-ix/donut-base", config=config)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [8]:
added_tokens = [fr'<{x}>' for x in type_dict.values()] + [fr'</{x}>' for x in type_dict.values()]
processor.tokenizer.add_tokens(added_tokens)
model.decoder.resize_token_embeddings(len(processor.tokenizer))
model.config.pad_token_id = processor.tokenizer.pad_token_id
model.config.decoder_start_token_id = processor.tokenizer.convert_tokens_to_ids(['<s>'])[0]

In [9]:
print("Pad token ID:", processor.decode([model.config.pad_token_id]))
print("Decoder start token ID:", processor.decode([model.config.decoder_start_token_id]))

Pad token ID: <pad>
Decoder start token ID: <s>


In [10]:
from sklearn.model_selection import train_test_split

train_val_df = pd.read_csv("./dataframes/train_annot_df.csv")
test_df = pd.read_csv("./dataframes/test_annot_df.csv")

train_df, val_df = train_test_split(train_val_df, test_size=0.2, random_state=CFG['SEED'])

train_dataset = DonutDataset(train_df, max_length=CFG['MAX_LEN'], processor=processor, split="train")
val_dataset = DonutDataset(val_df, max_length=CFG['MAX_LEN'], processor=processor, split="validation")
test_dataset = DonutDataset(test_df, max_length=CFG['MAX_LEN'], processor=processor, split="test")

**Dataloader Building**

In [11]:
train_dataloader = DataLoader(train_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=True, num_workers=CFG['NUM_WORKERS'])
val_dataloader = DataLoader(val_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=CFG['NUM_WORKERS'])
test_dataloader = DataLoader(test_dataset, batch_size=CFG['BATCH_SIZE'], shuffle=False, num_workers=CFG['NUM_WORKERS'])

In [12]:
#Batch Verifying
batch = next(iter(train_dataloader))
pixel_values, labels, target_sequences = batch
print(pixel_values.shape)

torch.Size([2, 3, 4032, 3024])


In [13]:
for id in labels[0].tolist()[:30]:
  if id != -100:
    print(processor.decode([id]))
  else:
    print(id)

<des>

락
티
케
어
</des>
<nm>
제
마
지
스
</nm>
<des>
로
션
0
.
25%

프
레
드
니
카
르
베
이
트
전문


In [14]:
print(target_sequences[0])

<des>락티케어</des><nm>제마지스</nm><des>로션 0.25% 프레드니카르베이트 전문의약품 본 상표는 GSK 그룹사 실시권을 [2017] GSK 허여 받은 그룹사 것입니다. 또는 소유이거나 실시권자 Stiefel 20g C</des>


In [ ]:
class DonutModelPLModule(pl.LightningModule):
    def __init__(self, config, processor, model):
        super().__init__()
        self.config = config
        self.processor = processor
        self.model = model

    def training_step(self, batch, batch_idx):
        pixel_values, labels, _ = batch
        
        outputs = self.model(pixel_values, labels=labels)
        loss = outputs.loss
        self.log("train_loss", loss)
        return loss

    def validation_step(self, batch, batch_idx, dataset_idx=0):
        pixel_values, labels, answers = batch
        batch_size = pixel_values.shape[0]
        # we feed the prompt to the model
        decoder_input_ids = torch.full((batch_size, 1), self.model.config.decoder_start_token_id, device=self.device)
        
        outputs = self.model.generate(pixel_values,
                                   decoder_input_ids=decoder_input_ids,
                                   max_length=max_length,
                                   early_stopping=True,
                                   pad_token_id=self.processor.tokenizer.pad_token_id,
                                   eos_token_id=self.processor.tokenizer.eos_token_id,
                                   use_cache=True,
                                   num_beams=1,
                                   bad_words_ids=[[self.processor.tokenizer.unk_token_id]],
                                   return_dict_in_generate=True,)
    
        predictions = []
        for seq in self.processor.tokenizer.batch_decode(outputs.sequences):
            seq = seq.replace(self.processor.tokenizer.eos_token, "").replace(self.processor.tokenizer.pad_token, "")
            seq = re.sub(r"<.*?>", "", seq, count=1).strip()  # remove first task start token
            predictions.append(seq)

        scores = []
        for pred, answer in zip(predictions, answers):
            pred = re.sub(r"(?:(?<=>) | (?=", "", answer, count=1)
            answer = answer.replace(self.processor.tokenizer.eos_token, "")
            scores.append(edit_distance(pred, answer) / max(len(pred), len(answer)))

            if self.config.get("verbose", False) and len(scores) == 1:
                print(f"Prediction: {pred}")
                print(f"    Answer: {answer}")
                print(f" Normed ED: {scores[0]}")

        self.log("val_edit_distance", np.mean(scores))
        
        return scores

    def configure_optimizers(self):
        # you could also add a learning rate scheduler if you want
        optimizer = torch.optim.Adam(self.parameters(), lr=self.config.get("lr"))
    
        return optimizer

    def train_dataloader(self):
        return train_dataloader

    def val_dataloader(self):
        return val_dataloader

In [ ]:
# Working in Progress...S